# Karakter Seviyesi Dil Modeli - Dinazor Adası



---


[<img align="right" width="100" height="100" src="http://www.i2symbol.com/images/symbols/style-letters/circled_latin_capital_letter_a_u24B6_icon_128x128.png">](https://www.ayyucekizrak.com/)


Daha önce toplanmış olan dinazor isimleri dinos.txt dosyasında tutulmaktadır. Bu veriler kullanılarak yeni dinazor isimleri üretmek için karakter seviyesinde dil modeli oluşturulacaktır. Algoritmanız veri kümenizdeki bazı örüntüleri öğrenerek rastgele yeni isimler üretecek.    

Bu uygulama ile şunları öğreneceksiniz.

- RNN ile işlenen metin verileri nasıl saklanır
- Karakter seviyesinde metin üreten RNN nasıl oluşturulur
- Gradyan kırpma neden önemlidir

İlk olarak kendi hazırladığımız `utils` dosyasından bazı fonksiyonları ve `numpy` kütüphanesini dahil ederek başlıyoruz. 

In [0]:
import numpy as np
from utils import *
import random

## 1 - Problem Tanımı

### 1.1 - Veri Kümesi ve Ön İşlemler

Dinazor isimlerinden oluşan veri kümesini okuyalım, a'dan z'ye karakterlerden oluşan listeler oluşturalım, veri kümesi ve oluşturduğumuz sözlüğün boyutlarını hesaplayalım.

In [0]:
data = open('dinos.txt', 'r').read()
data= data.lower()
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('Veriniz toplam %d karakterden ve %d tekil karakterden oluşmaktadır.' % (data_size, vocab_size))

Veriniz toplam 19911 karakterden ve 27 tekil karakterden oluşmaktadır.


Oluşturduğumuz sözlük 26 karakterden (a-z) ve 1 adet yeni satır karakterinden (\n) oluşmaktadır. Aşağıdaki hücrede her bir karakterin 1-26 arasında bir sayı karşılığının olduğu Python sözlüğü (dictionary) oluşturuyoruz. Aynı şekilde sayıdan karaktere dönüşümü sağlamak için ikinci bir sözlük oluşturuyoruz.

In [0]:
char_to_ix = { ch:i for i,ch in enumerate(sorted(chars)) }
ix_to_char = { i:ch for i,ch in enumerate(sorted(chars)) }
print(ix_to_char)

{0: '\n', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}


### 1.2 - Modele Genel Bir Bakış

Modelinizde şunlar olacak: 

- Parametreleri ilklendir 
- Optimizasyon döngüsü
    - Yitim fonksiyonunu hesaplamak için ileri yönlü hesaplama
    - Gradyan hesabı için geriye yayılım
    - Gradyanlar patlamasını önlemek için gradyanları kırp
    - Gradyanları kullanarak parametreleri güncelle
- Öğrenilen parametreleri döndür 
    
Her bir zaman adımında, RNN önceki karakterlere bağlı olarak bir sonraki karakterin ne olduğunu tahmin etmeye çalışır. Veri kümesi; $X = (x^{\langle 1 \rangle}, x^{\langle 2 \rangle}, ..., x^{\langle T_x \rangle})$ eğitim kümesinde bulunan karakter listelerinden ve $Y = (y^{\langle 1 \rangle}, y^{\langle 2 \rangle}, ..., y^{\langle T_x \rangle})$ her bir $t$ anındaki çıkış değerlerinden oluşmaktadır. Tahmin edebileceğiniz gibi $y^{\langle t \rangle} = x^{\langle t+1 \rangle}$ eşitliği bulunmaktadır. 

## 2 - Modele ait Blokları Oluşturma

Bu bölümde, modele ait çok önemli iki parçayı oluşturacaksınız: 

- Gradyan kırpma: Gradyan patlamasını önlemek için
- Örnekleme: karakter oluşturmada kullanılan yöntem

### 2.1 - Gradyan Kırpma

Bu bölümde optimizasyon döngüsü içerisinde `clip` fonksiyonunu tanımlayacağız. Genel döngünün maliyet hesabının yapıldığı ileri yönlü hesaplama ve parametre güncellemelerinin yapıldığı geriye yayılım adımlarından oluştuduğunu hatırlayın. Gradyan patlamasını yani gradyanların büyük değerler almasını önlemek için parametre güncelleme işleminden önce gradyan kırpma işlemi gerçekleştirilmektedir.

Aşağıda gradyanları giriş olarak alan ve ihtiyaç var ise çıktı olarak kırpılmış versiyonlarını döndüren `clip` fonksiyonunu tanımlayacağız. Gradyan kırpmanın birçok farklı yöntemi bulunmaktadır. Biz burada gradyan vektörünün her bir elemanını [-N, N] aralığına indirgeyeceğiz. Bunu gerçekleştirmek için `maxValue` isimli bir değişken tanımlayıp değerini örneğin 10 olarak belirleyeceğiz. Gradyan kırpma işlemind değerler 10'dan büyük ise yeni değer 10 olarak, -10'dan küçük ise yeni değer -10 olarak ayarlanacaktır. Yani gradyan değerleri hiç bir şekilde gradyanların mutlak değeri `maxValue` değerinden küçük olamaz.

In [0]:
def clip(gradients, maxValue):
    '''
    Gradyan değerlerini maksimum ve minimum değerleri arasında tutar.
    
    Giriş:
    gradients -- "dWaa", "dWax", "dWya", "db", "dby" gradyanlarını içeren gradyan vektörü
    maxValue -- gradyanların kırpılacağı değer 
    
    Çıkış: 
    gradients -- kırpılmış gradyan
    '''
    
    dWaa, dWax, dWya, db, dby = gradients['dWaa'], gradients['dWax'], gradients['dWya'], gradients['db'], gradients['dby']
   
    # gradyanları kırpalım [dWax, dWaa, dWya, db, dby]. 
    for gradient in [dWax, dWaa, dWya, db, dby]:
        gradients = np.clip(gradient, -10, maxValue, out=gradient)
    
    gradients = {"dWaa": dWaa, "dWax": dWax, "dWya": dWya, "db": db, "dby": dby}
    
    return gradients

In [0]:
np.random.seed(3)
dWax = np.random.randn(5,3)*10
dWaa = np.random.randn(5,5)*10
dWya = np.random.randn(2,5)*10
db = np.random.randn(5,1)*10
dby = np.random.randn(2,1)*10
gradients = {"dWax": dWax, "dWaa": dWaa, "dWya": dWya, "db": db, "dby": dby}
gradients = clip(gradients, 10)
print("gradients[\"dWaa\"][1][2] =", gradients["dWaa"][1][2])
print("gradients[\"dWax\"][3][1] =", gradients["dWax"][3][1])
print("gradients[\"dWya\"][1][2] =", gradients["dWya"][1][2])
print("gradients[\"db\"][4] =", gradients["db"][4])
print("gradients[\"dby\"][1] =", gradients["dby"][1])

gradients["dWaa"][1][2] = 10.0
gradients["dWax"][3][1] = -10.0
gradients["dWya"][1][2] = 0.2971381536101662
gradients["db"][4] = [10.]
gradients["dby"][1] = [8.45833407]


### 2.2 - Örnekleme

Modelinizin eğitildiğini varsayın. Yeni metinler oluşturmak istiyorsunuz.

`sample` ismindeki fonksiyonu kullanarak karakterleri 4 adımda örnekleyeceğiz:

- **Adım 1**: İlk önce sıfır vektörünü ağımıza giriş olarak verelim. $x^{\langle 1 \rangle} = \vec{0}$ Böylecek herhangi bir metin oluşturmadan önce varsayılan değerimiz bu giriş vektörü olacak. Ayrıca aktivasyon değerlerini de şu şekilde tanımlayalım: $a^{\langle 0 \rangle} = \vec{0}$

- **Adım 2**: $a^{\langle 1 \rangle}$ ve $\hat{y}^{\langle 1 \rangle}$ vektörlerini elde etmek için bir adım ileri yönlü hesaplama yapalım. Kullandığımız bazı denklemler:

$$ a^{\langle t+1 \rangle} = \tanh(W_{ax}  x^{\langle t \rangle } + W_{aa} a^{\langle t \rangle } + b)\tag{1}$$

$$ z^{\langle t + 1 \rangle } = W_{ya}  a^{\langle t + 1 \rangle } + b_y \tag{2}$$

$$ \hat{y}^{\langle t+1 \rangle } = softmax(z^{\langle t + 1 \rangle })\tag{3}$$

$\hat{y}^{\langle t+1 \rangle }$'in (softmax) olasılık vektörü olduğunu, değerlerinin 0-1 arasında olduğunun ve toplam değerin 1 olduğunu unutmayın. $\hat{y}^{\langle t+1 \rangle}_i$ değeri "i." bir sonraki karakterin olasığının göstermektedir. 

- **Adım 3**: Bu adımda örnekleme gerçekleştirilmektedir. $\hat{y}^{\langle t+1 \rangle }$ değerinin $\hat{y}^{\langle t+1 \rangle }_i = 0.16$ olması durumunda "i." karakteri %16 olasılıkla seçeceğiniz anlamına gelir.

- **Adım 4**: Bu adımda $x^{\langle t \rangle }$ değeri $x^{\langle t + 1 \rangle }$ değeriyle değiştirilmektedir.

In [0]:
def sample(parameters, char_to_ix, seed):
    """
    RNN olasılık çıkış değerlerine göre karakter dizisi örneklenir 

    Giriş:
    parameters -- Waa, Wax, Wya, by, ve b değerlerini içeren Python dictionary 
    char_to_ix -- Her bir karakteri indeks değerine haritalayan Python dictionary
    seed -- tekrarlanabilirlik için

    Çıkış:
    indices -- örneklenen karakterlerin uzunluğunu tutan liste
    """
    
    # "parameters" sözlüğünden parametreleri alalım
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    vocab_size = by.shape[0]
    n_a = Waa.shape[1]
    
    # Adım 1: ilk karakter için x one-hot vektörünü oluşturalım
    x = np.zeros((vocab_size,1))
    # Adım 1': "a_prev" değerlerini sıfır olarak ilklendirelim 
    a_prev = np.zeros((n_a,1))
    
    # boş bir indis listesi oluşturalım. 
    indices = []
    
    # Idx yeni satır karakteri tespit etmek için. -1 olarak ilklendirelim
    idx = -1 
    
    # t zaman adımlarında döngüyü gerçekleştirelim. 
    # her bir zaman adımında olasılık dağılımına göre örneklenen karakterin indisi "indices" listesine eklenir. 
    # 50 karaktere ulaşırsa durduracağız.
    # iyi eğitilmiş bir modelde 50 karakter çok fazla.

    counter = 0
    newline_character = char_to_ix['\n']
    
    while (idx != newline_character and counter != 50):
        
        # Adım 2: (1), (2) ve (3) denklemleri kullanılarak ileri yönlü hesaplama yapalım
        a = np.tanh(np.dot(Wax, x) + np.dot(Waa, a_prev)+ b) #b[:,0]
        z = np.dot(Wya,a) + by 
        y = softmax(z)
        
        np.random.seed(counter+seed) 
        
        # Adım 3: Olasılık dağılımından karakterlerin indekslerinin örneklenmesi
        idx = np.random.choice(range(vocab_size), p = y.ravel())
        
        # indeks değerini "indices"e ekleyelim
        indices.append(idx)
        
        # Adım 4: Giriş karakteri ile örneklenen indeks değerine karşılık gelen karakteri değiştirilir
        x =  np.zeros((vocab_size,1))
        x[idx] = 1
        
        # "a_prev" değerini "a" ile güncelleyelim
        a_prev = a
        
        seed += 1
        counter +=1
        
    if (counter == 50):
        indices.append(char_to_ix['\n'])
    
    return indices

In [0]:
np.random.seed(2)
_, n_a = 20, 100
Wax, Waa, Wya = np.random.randn(n_a, vocab_size), np.random.randn(n_a, n_a), np.random.randn(vocab_size, n_a)
b, by = np.random.randn(n_a, 1), np.random.randn(vocab_size, 1)
parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "b": b, "by": by}

indices = sample(parameters, char_to_ix, 0)
print("Örnekleme:")
print("örneklenen indis listesi:", indices)
print("örneklenen karakterler:", [ix_to_char[i] for i in indices])

Örnekleme:
örneklenen indis listesi: [12, 17, 24, 14, 13, 9, 10, 22, 24, 6, 13, 11, 12, 6, 21, 15, 21, 14, 3, 2, 1, 21, 18, 24, 7, 25, 6, 25, 18, 10, 16, 2, 3, 8, 15, 12, 11, 7, 1, 12, 10, 2, 7, 7, 11, 17, 24, 12, 19, 1, 0]
örneklenen karakterler: ['l', 'q', 'x', 'n', 'm', 'i', 'j', 'v', 'x', 'f', 'm', 'k', 'l', 'f', 'u', 'o', 'u', 'n', 'c', 'b', 'a', 'u', 'r', 'x', 'g', 'y', 'f', 'y', 'r', 'j', 'p', 'b', 'c', 'h', 'o', 'l', 'k', 'g', 'a', 'l', 'j', 'b', 'g', 'g', 'k', 'q', 'x', 'l', 's', 'a', '\n']


## 3 - Dil Modelini Oluşturma

Yeni metin oluşturabilmek için karakter seviyesi dil modelini oluşturalım.


### 3.1 - Gradyan Düşümü 

Bu bölümde stokastik gradyan düşümü algoritmasını bir adım gerçekleyen bir fonksiyonu tanımlayacağız. RNN için bir adım optimizasyon döngüsü şu şekilde:

    - Yitim fonksiyonunu hesaplamak için ileri yönlü hesaplama
    - Gradyan hesabı için geriye yayılım
    - Gradyanlar patlamasını önlemek için gradyanları kırp
    - Gradyanları kullanarak parametreleri güncelle

**Örneğin**: Optimizasyon işlemlerinin uygulanması (bir adım Gradyan İniş Algoritması). 



```python
def rnn_forward(X, Y, a_prev, parameters):
    """ İleri yyayılım performansı (RNN için) cross-entropyler hesaplanır ve 'cache' de tutulur."""
    ....
    return loss, cache
    
def rnn_backward(X, Y, parameters, cache):
    """Parametrelere göre kayıp gradyanlarını hesaplamak için zamanla geriye doğru yayılımı gerçekleştirir. Ayrıca tüm gizli durumları da döndürür."""
    ...
    return gradients, a

def update_parameters(parameters, gradients, learning_rate):
    """Gradient Descent Güncelleme Kuralını kullanarak güncellemeler parametreleri."""
    ...
    return parameters
```

In [0]:
def optimize(X, Y, a_prev, parameters, learning_rate = 0.01):
    """
    Modeli eğitmek için bir adım optimizasyon uygulayalım
    
    Girdiler:
    X -- tamsayı listesi, her bir sayı sözlükte bir karakteri belirtir
    Y -- tamsayı listesi, X ile aynı fakat sola doğru 1 indeks kaydırılmış hali
    a_prev -- geçmiş saklı durumlar
    parameters -- Python sözlüğü:
                        Wax -- Ağırlık matrisi ile giriş çaprımı, (n_a, n_x) boyutlu Numpy dizisi
                        Waa -- Ağırlık matiris ile saklı durum çarpımı, (n_a, n_a) boyutlu Numpy dizisi
                        Wya -- Saklı durumdan çıkışa ağırlık matrisi, (n_y, n_a) boyutlu Numpy dizisi
                        b --  Bias, (n_a, 1) boyutlu Numpy dizisi
                        by -- Saklı durumdan çıkışa Bias, (n_y, 1) boyutlu Numpy dizisi   
    
    learning_rate -- öğrenme oranı
    
    Çıktılar:
    loss -- yitim fonksiyonu değeri (cross-entropy)
    gradients -- python dictionary:
                        dWax -- Girişten gizli katmana ağırlık gradyanları, boyut: (n_a, n_x)
                        dWaa -- Gizliden gizli katmana ağırlık gradyanları, boyut: (n_a, n_a)
                        dWya -- Gizliden çıkışa ağırlık gradyanı, boyut: (n_y, n_a)
                        db -- Bias vektörünün gradyanı, boyut: (n_a, 1)
                        dby -- Çıkış bias vektörünün gradyanı, boyut: (n_y, 1)
    a[len(X)-1] -- en son saklı durum, boyut: (n_a, 1)
    """
        
    # zamanda ileri yayılım
    # ileri yönlü hesaplama yapılarak cross-entropy yitimi hesaplanır.
    # yitim değerinin yanında cache değerleri de döndürülür
    loss, cache = rnn_forward(X, Y, a_prev, parameters)
    
    # zamanda geri yayılım
    # gradyan hesaplaması yapılır.
    # saklı durumlar da döndürülür.    
    gradients, a = rnn_backward(X, Y, parameters, cache)
    
    # Gradyanları -5 (minimum) ve 5 (maksimum) arasında kırpalım
    gradients = clip(gradients, 5)
    
    # Parametre güncellemesi
    parameters = update_parameters(parameters, gradients, learning_rate)
        
    return loss, gradients, a[len(X)-1]

In [0]:
np.random.seed(1)
vocab_size, n_a = 27, 100
a_prev = np.random.randn(n_a, 1)
Wax, Waa, Wya = np.random.randn(n_a, vocab_size), np.random.randn(n_a, n_a), np.random.randn(vocab_size, n_a)
b, by = np.random.randn(n_a, 1), np.random.randn(vocab_size, 1)
parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "b": b, "by": by}
X = [12,3,5,11,22,3]
Y = [4,14,11,22,25, 26]

loss, gradients, a_last = optimize(X, Y, a_prev, parameters, learning_rate = 0.01)
print("Loss =", loss)
print("gradients[\"dWaa\"][1][2] =", gradients["dWaa"][1][2])
print("np.argmax(gradients[\"dWax\"]) =", np.argmax(gradients["dWax"]))
print("gradients[\"dWya\"][1][2] =", gradients["dWya"][1][2])
print("gradients[\"db\"][4] =", gradients["db"][4])
print("gradients[\"dby\"][1] =", gradients["dby"][1])
print("a_last[4] =", a_last[4])

Loss = 126.50397572165339
gradients["dWaa"][1][2] = 0.19470931534726663
np.argmax(gradients["dWax"]) = 93
gradients["dWya"][1][2] = -0.007773876032004546
gradients["db"][4] = [-0.06809825]
gradients["dby"][1] = [0.01538192]
a_last[4] = [-1.]


### 3.2 - Modelin Eğitimi 

Dinazor isimlerinden oluşan veri kümesinde her bir satır bir eğitim örneğidir. Her 100 stokastastik gradyan düşümü adımında, rastgele 10 örnek üretilir. Stokastik gradyan düşümünün örnekleri rastgele ziyaret etmek için veri kümesi karıştırılır.

In [0]:
def model(data, ix_to_char, char_to_ix, num_iterations = 35000, n_a = 50, dino_names = 7, vocab_size = 27):
    """
    Modeli eğitir ve dinazor isimleri üretir
    
    Girdiler:
    data -- metin corpus
    ix_to_char -- indeksten karaktere haritalama
    char_to_ix -- karakterden indekse haritalama
    num_iterations -- modelin eğitimi için iterasyon sayısı
    n_a -- RNN hücresindeki birim sayısı 
    dino_names -- her bir iterasyonda örneklenecek dinazor ismi sayısı 
    vocab_size -- metinde bulunan tekil karakter sayısı 
    
    Çıktı:
    parameters -- öğrenilen parametreler
    """
    
    # vocab_size'dan n_x ve n_y değerlerini alalım 
    n_x, n_y = vocab_size, vocab_size
    
    # parametreleri ilklendirelim
    parameters = initialize_parameters(n_a, n_x, n_y)
    
    # yitim fonksiyonunu ilklendirelim
    loss = get_initial_loss(vocab_size, dino_names)
    
    # eğitim örneklerini kullanarak dinazor isimlerini listeleyelim
    with open("dinos.txt") as f:
        examples = f.readlines()
    examples = [x.lower().strip() for x in examples]
    
    # Dinazor isimlerini karıştıralım
    np.random.seed(0)
    np.random.shuffle(examples)
    
    # LSTM gizli durumunu (hidden state) ilklendirelim
    a_prev = np.zeros((n_a, 1))
    
    # Optimizasyon döngüsü
    for j in range(num_iterations):
            
        index = j % len(examples)
        X = [None] + [char_to_ix[ch] for ch in examples[index]] 
        Y = X[1:] + [char_to_ix["\n"]]
        
        # Optimizasyon döngüsü: ileri hesaplama -> geriye yayılım -> kırpma -> parametre güncelle
        # Öğrenme oranını 0.01 olarak seçelim
        curr_loss, gradients, a_prev = optimize(X, Y, a_prev, parameters, learning_rate = 0.01)
                
        # latency trick kullanarak yitimini düzleştirelim (smooth). Eğitimi hızlandırır.
        loss = smooth(loss, curr_loss)

        # Her 2000 iterasyonda sample() fonksiyonu ile "n" karakter oluşturalım
        # modelin iyi eğitildiği kontrol edilir
        if j % 2000 == 0:
            
            print('İterasyon: %d, Yitim: %f' % (j, loss) + '\n')
            
            # Ekrana yazdırılacak dinazor ismi sayısı
            seed = 0
            for name in range(dino_names):
                
                # Örneklenen indisler 
                sampled_indices = sample(parameters, char_to_ix, seed)
                print_sample(sampled_indices, ix_to_char)
                
                seed += 1
      
            print('\n')
        
    return parameters

Üstteki hücreyi çalıştırdığınızda modelinizin ilk önce rastgele karakterler daha sonra ise makul isimler ürettiğini gözlemleyeceksiniz.

In [0]:
parameters = model(data, ix_to_char, char_to_ix)

İterasyon: 0, Yitim: 23.087336

Nkzxwtdmfqoeyhsqwasjkjvu
Knfb
Kzxwtdmfqoeyhsqwasjkjvu
Nfb
Zxwtdmfqoeyhsqwasjkjvu
Fb
Xwtdmfqoeyhsqwasjkjvu


İterasyon: 2000, Yitim: 27.967193

Livtosaurus
Hlca
Hwtosaurus
Lacahosanaus
Xusalncoolurus
A
Tosaurus


İterasyon: 4000, Yitim: 25.730191

Livosaurus
Hmba
Iusosaurus
Ledalosaurus
Xusmaneroptosaurus
Baagosaurus
Torapgosaurus


İterasyon: 6000, Yitim: 24.635574

Onyusalonosaurus
Lleeacosaurus
Lyusqcheromus
Olaberriceptosaurus
Xusteolosaurus
Edalosaurus
Trogonosaurus


İterasyon: 8000, Yitim: 24.221203

Niushuhelnomus
Jia
Kutrodhchiaterauaurus
Nca
Xutokellolus
Caagropaboter
Trodhcholus


İterasyon: 10000, Yitim: 23.775948

Niwrosaurus
Kiacalosaurus
Lustoinatopteriuaneonsharriyilanbesaurus
Necairona
Xtrocheorax
Caadosaurus
Trocheorax


İterasyon: 12000, Yitim: 23.310353

Onyusaorashevasaurus
Liacalosaurus
Lustrapdoraxaurochlanosaurus
Olaalosaurus
Xussanchudyanitasaurus
Daakrolabnsaurosqracxalosaurus
Strapanlaverataramnptaronylealaluakachiachatolonha




## Sonuç

Görüldüğü üzere algoritmanız eğitimin sonlarına doğru daha iyi dinazor isimleri üretmeyi başardı. <img align="right" width="300" height="150" src="https://www.yayomg.com/wp-content/uploads/2016/02/yayomg-good-dinosaur-life-lessons-7.gif">

İlk başlarda rastgele karakter dizileri oluşturulurken sonlara doğru daha makul isimler üretilmeye başlandı. 


**`maconucon`**, **`marloralus`** and **`macingsersaurus`** gibi isimler üretilmiştir.


Geliştirdiğimiz model dinazor isimlerinin **`saurus`**, **`don`**, **`aura`**, **`tor`** vb. ile bittiğini öğrenmiştir.

